<a href="https://colab.research.google.com/github/emolinaperez/econometrics_mek/blob/main/Week%208/R/Week8_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 8: Econometrics

In [1]:
#load library
if (!require(pacman)) install.packages("pacman")
library(pacman)
p_load('car','modelsummary')

Loading required package: pacman

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pacman’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘Rdpack’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’



car installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bayestestR’, ‘datawizard’, ‘checkmate’, ‘insight’, ‘parameters’, ‘performance’, ‘tables’, ‘tinytable’



modelsummary installed



In [3]:
#Read data
data <- read.table("https://raw.githubusercontent.com/emolinaperez/econometrics_mek/main/Week%208/data/MRW1992.txt",header=TRUE)
head(data)

,country,N,I,O,Y60,Y85,Y_growth,pop_growth,invest,school
,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Algeria,1,1,0,2485,4371,4.8,2.6,24.1,4.5
2,Angola,1,0,0,1588,1171,0.8,2.1,5.8,1.8
3,Benin,1,0,0,1116,1071,2.2,2.4,10.8,1.8
4,Botswana,1,1,0,959,3671,8.6,3.2,28.3,2.9
5,BurkinaFaso,1,0,0,529,857,2.9,0.9,12.7,0.4
6,Burundi,1,0,0,755,663,1.2,1.7,5.1,0.4


In [4]:
N <- matrix(data$N,ncol=1)
lndY <- matrix(log(data$Y85)-log(data$Y60),ncol=1)
lnY60 <- matrix(log(data$Y60),ncol=1)
lnI <- matrix(log(data$invest/100),ncol=1)
lnG <- matrix(log(data$pop_growth/100+0.05),ncol=1)
lnS <- matrix(log(data$school/100),ncol=1)
xx <- as.matrix(cbind(lnY60,lnI,lnG,lnS,matrix(1,nrow(lndY),1)))
x <- xx[N==1,]
y <- lndY[N==1]
n <- nrow(x)
k <- ncol(x)

In [5]:
#Unrestricted regression
invx <-solve(t(x)%*%x)
b_ols <- solve((t(x)%*%x),(t(x)%*%y))
e_ols <- rep((y-x%*%b_ols),times=k)
xe_ols <- x*e_ols
V_ols <- (n/(n-k))*invx%*%(t(xe_ols)%*%xe_ols)%*%invx
se_ols <- sqrt(diag(V_ols))
print(b_ols)
print(se_ols)

           [,1]
[1,] -0.2883737
[2,]  0.5237367
[3,] -0.5056565
[4,]  0.2311171
[5,]  3.0215222
[1] 0.05427556 0.10729137 0.23603269 0.06640414 0.73730944


In [6]:
#Constrained regression
R <- c(0,1,1,1,0)
iR <- invx%*%R%*%solve(t(R)%*%invx%*%R)%*%t(R)
b_cls <- b_ols - iR%*%b_ols
e_cls <- rep((y-x%*%b_cls),times=k)
xe_cls <- x*e_cls
V_tilde <- (n/(n-k+1))*invx%*%(t(xe_cls)%*%xe_cls)%*%invx
V_cls <- V_tilde - iR%*%V_tilde - V_tilde%*%t(iR) +iR%*%V_tilde%*%t(iR)
se_cls <- sqrt(diag(V_cls))
print(b_cls)
print(se_cls)

           [,1]
[1,] -0.2979013
[2,]  0.5006704
[3,] -0.7358562
[4,]  0.2351858
[5,]  2.4569123
[1] 0.05278963 0.09183351 0.07769231 0.06500286 0.43902622


In [7]:
#Efficient minimum distance
Vr <- V_ols%*%R%*%solve(t(R)%*%V_ols%*%R)%*%t(R)
b_emd <- b_ols - Vr%*%b_ols
e_emd <- rep((y-x%*%b_emd),times=k)
xe_emd <- x*e_emd
V2 <- (n/(n-k+1))*invx%*%(t(xe_emd)%*%xe_emd)%*%invx
V_emd <- V2 - V2%*%R%*%solve(t(R)%*%V2%*%R)%*%t(R)%*%V2

In [8]:
#with linear models
data$Y <- log(data$Y85)
data$GDP_1960 <- log(data$Y60)
data$I_GDP <-  log(data$invest/100)
data$n_g_s <- log(data$pop_growth/100+0.05)
data$log_school <- log(data$school/100)

In [9]:
# Unrestricted regression
model.u <- lm( Y ~ I_GDP + n_g_s , data=subset(data,N==1))
summary(model.u)


Call:
lm(formula = Y ~ I_GDP + n_g_s, data = subset(data, N == 1))

Residuals:
     Min       1Q   Median       3Q      Max 
-1.79144 -0.39367  0.04124  0.43368  1.58046 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   5.4299     1.5839   3.428 0.000900 ***
I_GDP         1.4240     0.1431   9.951  < 2e-16 ***
n_g_s        -1.9898     0.5634  -3.532 0.000639 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6891 on 95 degrees of freedom
Multiple R-squared:  0.6009,	Adjusted R-squared:  0.5925 
F-statistic: 71.51 on 2 and 95 DF,  p-value: < 2.2e-16


In [10]:
# Restricted regression
model.r <- lm( Y ~ I_GDP + n_g_s , data=subset(data,N==1))
summary(model.r)


Call:
lm(formula = Y ~ I_GDP + n_g_s, data = subset(data, N == 1))

Residuals:
     Min       1Q   Median       3Q      Max 
-1.79144 -0.39367  0.04124  0.43368  1.58046 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   5.4299     1.5839   3.428 0.000900 ***
I_GDP         1.4240     0.1431   9.951  < 2e-16 ***
n_g_s        -1.9898     0.5634  -3.532 0.000639 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6891 on 95 degrees of freedom
Multiple R-squared:  0.6009,	Adjusted R-squared:  0.5925 
F-statistic: 71.51 on 2 and 95 DF,  p-value: < 2.2e-16


In [11]:
# test of restriction
car::linearHypothesis(model.r, "I_GDP + n_g_s = 0")

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,96,45.50375,NA,NA,NA,NA
2,95,45.10763,1,0.3961221,0.8342623,0.3633552


In [12]:
# p-value
car::linearHypothesis(model.r, "I_GDP + n_g_s = 0")$`Pr(>F)`[2]

[1] 0.3633552

Implied α

The "Implied α" (implied alpha) shown in the Mankiw, Romer and Weil table refers to the share of capital in the Cobb-Douglas production function of the Solow model.


In [13]:
# alpha
alpha <- coef(model.r)[2] / (1 + coef(model.r)[2])
alpha

I_GDP 
0.5874612

## Table 1: ESTIMATION SOLOW OF THE TEXTBOOK MODE

### Unrestricted regression

In [14]:
# Non-oil, unrestricted regression
model1a <- lm(log(Y85) ~ I_GDP + n_g_s,
             data=subset(data, N==1))

# Intermediate, unrestricted regression
model1b <- lm(log(Y85) ~ I_GDP + n_g_s,
             data=subset(data, I==1))

# OECD, unrestricted regression
model1c <- lm(log(Y85) ~ I_GDP + n_g_s,
             data=subset(data, O==1))

In [16]:
# Table 1 - Unrestricted regression
modelsummary(list(model1a, model1b, model1c),
             stars = c('*' = .1, '**' = .05, '***' = .01),
             gof_map = c("nobs", "r.squared"),
             coef_rename = c('(Intercept)' = "CONSTANT",
                             'I_GDP' = 'ln(I/GDP)',
                             'n_g_s' = 'ln(n + g + δ)'))


+---------------+-----------+-----------+----------+
|               | (1)       | (2)       | (3)      |
+===============+===========+===========+==========+
| CONSTANT      | 5.430***  | 5.346***  | 8.021*** |
+---------------+-----------+-----------+----------+
|               | (1.584)   | (1.543)   | (2.518)  |
+---------------+-----------+-----------+----------+
| ln(I/GDP)     | 1.424***  | 1.318***  | 0.500    |
+---------------+-----------+-----------+----------+
|               | (0.143)   | (0.171)   | (0.434)  |
+---------------+-----------+-----------+----------+
| ln(n + g + δ) | -1.990*** | -2.017*** | -0.742   |
+---------------+-----------+-----------+----------+
|               | (0.563)   | (0.534)   | (0.852)  |
+---------------+-----------+-----------+----------+
| Num.Obs.      | 98        | 75        | 22       |
+---------------+-----------+-----------+----------+
| R2            | 0.601     | 0.599     | 0.106    |
+===============+===========+===========+====

### Restricted regression (ln(I/GDP) - ln(n + g )+ δ)

In [17]:
# Non-oil, restricted regression
model2a <- lm(log(Y85) ~ I(I_GDP - n_g_s),
             data=subset(data, N==1))

# Intermediate, restricted regression
model2b <- lm(log(Y85) ~ I(I_GDP - n_g_s),
             data=subset(data, I==1))

# OECD, restricted regression
model2c <- lm(log(Y85) ~ I(I_GDP - n_g_s),
             data=subset(data, O==1))

In [18]:
# Table 1 - Restricted regression
modelsummary(list(model2a, model2b, model2c),
             stars = c('*' = .1, '**' = .05, '***' = .01),
             gof_map = c("nobs", "r.squared"),
             coef_rename = c('(Intercept)' = "CONSTANT",
                             'I(I_GDP - n_g_s)' = 'ln(I/GDP) - ln(n + g + δ)'))


+---------------------------+----------+----------+----------+
|                           | (1)      | (2)      | (3)      |
+===========================+==========+==========+==========+
| CONSTANT                  | 6.872*** | 7.093*** | 8.624*** |
+---------------------------+----------+----------+----------+
|                           | (0.121)  | (0.146)  | (0.533)  |
+---------------------------+----------+----------+----------+
| ln(I/GDP) - ln(n + g + δ) | 1.488*** | 1.431*** | 0.554    |
+---------------------------+----------+----------+----------+
|                           | (0.125)  | (0.139)  | (0.365)  |
+---------------------------+----------+----------+----------+
| Num.Obs.                  | 98       | 75       | 22       |
+---------------------------+----------+----------+----------+
| R2                        | 0.597    | 0.592    | 0.103    |
+===========================+==========+==========+==========+
| * p < 0.1, ** p < 0.05, *** p < 0.01                

In [19]:
# test of restriction
test_a <- car::linearHypothesis(model1a, "I_GDP + n_g_s = 0")
test_b <- car::linearHypothesis(model1b, "I_GDP + n_g_s = 0")
test_c <- car::linearHypothesis(model1c, "I_GDP + n_g_s = 0")

In [20]:
# p-values
p_value_a <- test_a$`Pr(>F)`[2]
p_value_b <- test_b$`Pr(>F)`[2]
p_value_c <- test_c$`Pr(>F)`[2]

In [21]:
# Implied α
alpha_a <- coef(model2a)[2] / (1 + coef(model2a)[2])
alpha_b <- coef(model2b)[2] / (1 + coef(model2b)[2])
alpha_c <- coef(model2c)[2] / (1 + coef(model2c)[2])

In [22]:
test_rest <- data.frame(
  Sample = c("Non-oil", "Intermediate", "OECD"),
  p_value = c(p_value_a, p_value_b, p_value_c),
  Implied_alpha = c(alpha_a, alpha_b, alpha_c)
)

test_rest

Sample,p_value,Implied_alpha
<chr>,<dbl>,<dbl>
Non-oil,0.3633552,0.5980698
Intermediate,0.2592063,0.5886391
OECD,0.8085913,0.3564350


## Table 2: ESTIMATION SOLOW OF THE AUGMENTED MODE

In [23]:
# Non-oil
model3a <- lm(log(Y85) ~ I_GDP + n_g_s + log_school, data=subset(data, N==1))
model4a <- lm(log(Y85) ~ I(I_GDP - n_g_s) + I(log_school - n_g_s), data=subset(data, N==1))

# Intermediate
model3b <- lm(log(Y85) ~ I_GDP + n_g_s + log_school, data=subset(data, I==1))
model4b <- lm(log(Y85) ~ I(I_GDP - n_g_s) + I(log_school - n_g_s), data=subset(data, I==1))

# OECD
model3c <- lm(log(Y85) ~ I_GDP + n_g_s + log_school, data=subset(data, O==1))
model4c <- lm(log(Y85) ~ I(I_GDP - n_g_s) + I(log_school - n_g_s), data=subset(data, O==1))


In [24]:
# Table 2 - Unrestricted regression
modelsummary(list(model3a, model3b, model3c),
             stars = c('*' = .1, '**' = .05, '***' = .01),
             gof_map = c("nobs", "r.squared"),
             coef_rename = c('(Intercept)' = "CONSTANT",
                             'I_GDP' = 'ln(I/GDP)',
                             'n_g_s' = 'ln(n + g + δ)',
                             'log_school' = 'ln(SCHOOL)'))


+---------------+-----------+-----------+----------+
|               | (1)       | (2)       | (3)      |
+===============+===========+===========+==========+
| CONSTANT      | 6.844***  | 7.791***  | 8.637*** |
+---------------+-----------+-----------+----------+
|               | (1.177)   | (1.192)   | (2.214)  |
+---------------+-----------+-----------+----------+
| ln(I/GDP)     | 0.697***  | 0.700***  | 0.276    |
+---------------+-----------+-----------+----------+
|               | (0.133)   | (0.151)   | (0.389)  |
+---------------+-----------+-----------+----------+
| ln(n + g + δ) | -1.745*** | -1.500*** | -1.076   |
+---------------+-----------+-----------+----------+
|               | (0.416)   | (0.403)   | (0.756)  |
+---------------+-----------+-----------+----------+
| ln(SCHOOL)    | 0.654***  | 0.731***  | 0.768**  |
+---------------+-----------+-----------+----------+
|               | (0.073)   | (0.095)   | (0.293)  |
+---------------+-----------+-----------+----

In [25]:
# Table 2 - Restricted regression
modelsummary(list(model4a, model4b, model4c),
             stars = c('*' = .1, '**' = .05, '***' = .01),
             gof_map = c("nobs", "r.squared"),
             coef_rename = c('(Intercept)' = "CONSTANT",
                             'I(I_GDP - n_g_s)' = 'ln(I/GDP) - ln(n + g + δ)',
                             'I(log_school - n_g_s)' = 'ln(SCHOOL) - ln(n + g + δ)'))


+----------------------------+----------+----------+----------+
|                            | (1)      | (2)      | (3)      |
+============================+==========+==========+==========+
| CONSTANT                   | 7.853*** | 7.966*** | 8.716*** |
+----------------------------+----------+----------+----------+
|                            | (0.140)  | (0.154)  | (0.466)  |
+----------------------------+----------+----------+----------+
| ln(I/GDP) - ln(n + g + δ)  | 0.738*** | 0.709*** | 0.283    |
+----------------------------+----------+----------+----------+
|                            | (0.124)  | (0.138)  | (0.334)  |
+----------------------------+----------+----------+----------+
| ln(SCHOOL) - ln(n + g + δ) | 0.657*** | 0.733*** | 0.769**  |
+----------------------------+----------+----------+----------+
|                            | (0.073)  | (0.093)  | (0.284)  |
+----------------------------+----------+----------+----------+
| Num.Obs.                   | 98      

## Table 3: TESTS FOR UNCONDITIONAL CONVERGE

In [26]:
# Non-oil
model5a <- lm(log(Y85)-log(Y60) ~ log(Y60) , data=subset(data, N==1))

# Intermediate
model5b <- lm(log(Y85)-log(Y60) ~ log(Y60) , data=subset(data, I==1))

# OECD
model5c <- lm(log(Y85)-log(Y60) ~ log(Y60) , data=subset(data, O==1))


In [27]:
# Table 3
modelsummary(list(model5a, model5b, model5c),
             stars = c('*' = .1, '**' = .05, '***' = .01),
             gof_map = c("nobs", "r.squared"),
             coef_rename = c('(Intercept)' = "CONSTANT",
                             'log(Y60)' = 'ln(Y60)'))


+----------+---------+---------+-----------+
|          | (1)     | (2)     | (3)       |
+==========+=========+=========+===========+
| CONSTANT | -0.267  | 0.588   | 3.686***  |
+----------+---------+---------+-----------+
|          | (0.380) | (0.433) | (0.685)   |
+----------+---------+---------+-----------+
| ln(Y60)  | 0.094*  | -0.004  | -0.341*** |
+----------+---------+---------+-----------+
|          | (0.050) | (0.055) | (0.079)   |
+----------+---------+---------+-----------+
| Num.Obs. | 98      | 75      | 22        |
+----------+---------+---------+-----------+
| R2       | 0.036   | 0.000   | 0.485     |
+==========+=========+=========+===========+
| * p < 0.1, ** p < 0.05, *** p < 0.01     |
+==========+=========+=========+===========+ 

In [28]:
# Implied λ

# coefficient for log(Y60)
beta_a <- coef(model5a)[2]
T <- 25  # Time period 1985-1960

# calculate lambda (convergence rate)
lambda_a <- -(-log(1 - beta_a) / T)

cat(sprintf("Non-oil Implied λ: %.5f\n", lambda_a))


Non-oil Implied λ: -0.00396
